## Phishing Site Detection

In the Pre processing of the Data the attributes are coded into following format for application of KNN Classifier 

 	SFH's type is nominal, range is ('1', '-1', '0')
 	popUpWidnow's type is nominal, range is ('-1', '0', '1')
 	SSLfinal_State's type is nominal, range is ('1', '-1', '0')
 	Request_URL's type is nominal, range is ('-1', '0', '1')
 	URL_of_Anchor's type is nominal, range is ('-1', '0', '1')
 	web_traffic's type is nominal, range is ('1', '0', '-1')
 	URL_Length's type is nominal, range is ('1', '-1', '0')
 	age_of_domain's type is nominal, range is ('1', '-1')
 	having_IP_Address's type is nominal, range is ('0', '1')
 	Result's type is nominal, range is ('0', '1', '-1'))
    
    The  data coding is as follows:
    "1"  - Legitimate Website
    "0"  - Suspicious
    "-1" - Malicious Website

In [49]:
your_local_path = 'C:/Users/ohm/Downloads/Machine Learning Exercises/'

In [50]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from scipy.io.arff import loadarff
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [51]:
phishing_data_raw = loadarff(your_local_path+"PhishingData.arff")

In [52]:
phishing_data_raw

(array([(b'1', b'-1', b'1', b'-1', b'-1', b'1', b'1', b'1', b'0', b'0'),
        (b'-1', b'-1', b'-1', b'-1', b'-1', b'0', b'1', b'1', b'1', b'1'),
        (b'1', b'-1', b'0', b'0', b'-1', b'0', b'-1', b'1', b'0', b'1'),
        ...,
        (b'-1', b'0', b'-1', b'-1', b'-1', b'0', b'-1', b'-1', b'0', b'1'),
        (b'0', b'0', b'1', b'0', b'0', b'0', b'-1', b'1', b'0', b'1'),
        (b'1', b'0', b'1', b'1', b'1', b'0', b'-1', b'-1', b'0', b'-1')],
       dtype=[('SFH', 'S2'), ('popUpWidnow', 'S2'), ('SSLfinal_State', 'S2'), ('Request_URL', 'S2'), ('URL_of_Anchor', 'S2'), ('web_traffic', 'S2'), ('URL_Length', 'S2'), ('age_of_domain', 'S2'), ('having_IP_Address', 'S1'), ('Result', 'S2')]),
 Dataset: Phishing
 	SFH's type is nominal, range is ('1', '-1', '0')
 	popUpWidnow's type is nominal, range is ('-1', '0', '1')
 	SSLfinal_State's type is nominal, range is ('1', '-1', '0')
 	Request_URL's type is nominal, range is ('-1', '0', '1')
 	URL_of_Anchor's type is nominal, range is ('-1',

In [53]:
# Selection of the Data and converting data into numpy format for flexibility in cleaning
phising_data_array = np.array(phishing_data_raw[0])
phising_data_array

array([(b'1', b'-1', b'1', b'-1', b'-1', b'1', b'1', b'1', b'0', b'0'),
       (b'-1', b'-1', b'-1', b'-1', b'-1', b'0', b'1', b'1', b'1', b'1'),
       (b'1', b'-1', b'0', b'0', b'-1', b'0', b'-1', b'1', b'0', b'1'),
       ...,
       (b'-1', b'0', b'-1', b'-1', b'-1', b'0', b'-1', b'-1', b'0', b'1'),
       (b'0', b'0', b'1', b'0', b'0', b'0', b'-1', b'1', b'0', b'1'),
       (b'1', b'0', b'1', b'1', b'1', b'0', b'-1', b'-1', b'0', b'-1')],
      dtype=[('SFH', 'S2'), ('popUpWidnow', 'S2'), ('SSLfinal_State', 'S2'), ('Request_URL', 'S2'), ('URL_of_Anchor', 'S2'), ('web_traffic', 'S2'), ('URL_Length', 'S2'), ('age_of_domain', 'S2'), ('having_IP_Address', 'S1'), ('Result', 'S2')])

In [54]:
# Converting the numpy array into Pandas data frame  and casting the coloumns to numeric type
phising_data_frame = pd.DataFrame(phising_data_array).apply(pd.to_numeric)
print(phising_data_frame.shape)
phising_data_frame.head()

(1353, 10)


,SFH,popUpWidnow,SSLfinal_State,Request_URL,URL_of_Anchor,web_traffic,URL_Length,age_of_domain,having_IP_Address,Result
0,1,-1,1,-1,-1,1,1,1,0,0
1,-1,-1,-1,-1,-1,0,1,1,1,1
2,1,-1,0,0,-1,0,-1,1,0,1
3,1,0,1,-1,-1,0,1,1,0,0
4,-1,-1,1,-1,0,0,-1,1,0,1


In [55]:
col_names = list(phising_data_frame)# Geeting Column names of the pandas data frame
print("Coloumn_names:" + ", ".join(col_names))
# The scikit learn package takes data in form of labels and predictors
# Extracting Result coloum to pass as target coloum
labels = phising_data_frame["Result"].values
print(labels)

Coloumn_names:SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain, having_IP_Address, Result
[ 0  1  1 ...  1  1 -1]


In [56]:
predictor_cols = col_names[0:len(col_names)-2]# selecting the predictor column names
print("Predictor_Coloumns:" + ", ".join(predictor_cols))
#extracting Predictor points 
predictors = phising_data_frame[predictor_cols].values
predictors

Predictor_Coloumns:SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain


array([[ 1, -1,  1, ...,  1,  1,  1],
       [-1, -1, -1, ...,  0,  1,  1],
       [ 1, -1,  0, ...,  0, -1,  1],
       ...,
       [-1,  0, -1, ...,  0, -1, -1],
       [ 0,  0,  1, ...,  0, -1,  1],
       [ 1,  0,  1, ...,  0, -1, -1]], dtype=int64)

In [57]:
col_names_ml = list(phising_data_frame)# Geeting Column names of the pandas data frame
print("Coloumn_names:" + ", ".join(col_names_ml))
# The scikit learn package takes data in form of labels and predictors
# Extracting Result coloum to pass as target coloum
labels_ml = phising_data_frame["Result"].values
print(labels_ml)



predictor_cols_ml = col_names_ml[0:len(col_names_ml)-2]# selecting the predictor column names
print("Predictor_Coloumns:" + ", ".join(predictor_cols_ml))
#extracting Predictor points 
predictors_ml = phising_data_frame[predictor_cols_ml].values
print(predictors_ml)

Coloumn_names:SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain, having_IP_Address, Result
[ 0  1  1 ...  1  1 -1]
Predictor_Coloumns:SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain
[[ 1 -1  1 ...  1  1  1]
 [-1 -1 -1 ...  0  1  1]
 [ 1 -1  0 ...  0 -1  1]
 ...
 [-1  0 -1 ...  0 -1 -1]
 [ 0  0  1 ...  0 -1  1]
 [ 1  0  1 ...  0 -1 -1]]


In [58]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(predictors_ml, labels_ml, test_size=0.3, random_state=42)

#gridsearchCV for K (number of neighbors)
k = np.arange(1,31,1)# As a thumb rule k might be taken as squareroot of 'n', but gridsearch is the best option
parameters = {'n_neighbors' : k}
clf_knn = KNeighborsClassifier(weights = 'distance')
clf = GridSearchCV(clf_knn,parameters,cv=5)
clf_knn = clf.fit(X_train,y_train)
print(clf.best_params_)


#Prediction
pred = clf_knn.predict(X_test)
target_names = ['-1','0','1']
print(classification_report(y_test,pred,target_names=target_names))
confusion_matrix(y_test,pred)

{'n_neighbors': 4}
             precision    recall  f1-score   support

         -1       0.89      0.93      0.91       203
          0       0.83      0.83      0.83        35
          1       0.89      0.85      0.87       168

avg / total       0.89      0.89      0.89       406



array([[188,   3,  12],
       [  1,  29,   5],
       [ 22,   3, 143]], dtype=int64)

In [59]:
print(accuracy_score(y_test,pred))

0.8866995073891626


In [60]:
# with our model, we achieved 88.67% accuracy. By finetuning the hyper parameters of KNN Classifier, we can achieve more accuracy.